In [ ]:
#Imports
import numpy as np
import pandas as pd

import time
import gzip
import warnings
warnings.filterwarnings("ignore")

import cPickle

from sklearn.model_selection import train_test_split
import lightgbm as lgb

np.random.seed(669)

In [ ]:
X_tr = pd.read_csv('Unused/train_june2015_full.csv')
y_tr = pd.read_csv('train_june2015_labels.csv').values.ravel()

X_val = pd.read_csv('Unused/val_may2016_full.csv')
y_val = pd.read_csv('val_may2016_labels.csv').values.ravel()
y_val_prevs = pd.read_csv('val_may2016_previousproducts.csv').values
y_val_mapk = cPickle.load(open('val_may2016_mapklabelspickle.p', 'rb'))

X_tr.columns = clean + owned4 + owned3 + owned2 + owned1 + owned0
X_val.columns = clean + owned4 + owned3 + owned2 + owned1 + owned0

add_tr = pd.read_csv('FULL_train_june2015_5lags.csv').loc[:, ['age', 'antiguedad', 'renta']]
add_val = pd.read_csv('FULL_val_may2016_5lags.csv').loc[:, ['age', 'antiguedad', 'renta']]

add_tr.columns = ['age2', 'antiguedad2', 'renta2']
add_val.columns = ['age2', 'antiguedad2', 'renta2']

add_tr2 = pd.read_csv('FULL_train_june2015_5lags.csv').loc[:, ['fecha_alta']]
add_val2 = pd.read_csv('FULL_val_may2016_5lags.csv').loc[:, ['fecha_alta']]


In [ ]:
cats = [x for x in X_tr.columns if 'ult1' not in x]
cats.remove('indrel_1mes')
prods = [x for x in X_tr.columns if 'ult1' in x]
change = [x for x in X_tr.columns if 'change' in x]

orig_prods = prods[0:24]
prods = prods[24:]

lag1 = [x for x in X_tr.columns if '_4' in x]
lag2 = [x for x in X_tr.columns if '_3' in x]
lag3 = [x for x in X_tr.columns if '_2' in x]
lag4 = [x for x in X_tr.columns if '_1' in x][2:]
lag5 = [x for x in X_tr.columns if '_0' in x]

In [ ]:
X_tr.drop(orig_prods, 1, inplace = True)
X_val.drop(orig_prods, 1, inplace = True)

In [ ]:

lags2 = lag1 + lag2 + lag3 + lag4 + lag5
lags = [lag1, lag2, lag3, lag4, lag5]

def get_diffs(X1):
    
    X_tr = X1.copy()

    diff2 = pd.DataFrame(X_tr[lag1].values - X_tr[lag2].values)
    sum2 = pd.DataFrame(X_tr[lag1].values + X_tr[lag2].values)
    prod2 = pd.DataFrame(X_tr[lag1].values * X_tr[lag2].values)

    diff3 = pd.DataFrame(X_tr[lag2].values - X_tr[lag3].values)
    sum3 = pd.DataFrame(X_tr[lag2].values + X_tr[lag3].values)
    prod3 = pd.DataFrame(X_tr[lag2].values * X_tr[lag3].values)

    diff4 = pd.DataFrame(X_tr[lag3].values - X_tr[lag4].values)
    sum4 = pd.DataFrame(X_tr[lag3].values + X_tr[lag4].values)
    prod4 = pd.DataFrame(X_tr[lag3].values * X_tr[lag4].values)
    
    diff1 = pd.DataFrame(X_tr[lag4].values - X_tr[lag5].values)
    sum1 = pd.DataFrame(X_tr[lag4].values + X_tr[lag5].values)
    prod1 = pd.DataFrame(X_tr[lag4].values * X_tr[lag5].values)
    
    X_tr = pd.concat([X_tr, sum2, sum3, sum4, sum1, 
                      diff2, diff3, diff4, diff1,
                      prod2, prod3, prod4, prod1], 1)
    
    return X_tr


def add(X1):
    
    X = X1.copy()
    
    X['sum_cats'] = X[cats].sum(axis = 1)
    X['mean_cats'] = X[cats].mean(axis = 1)
    X['std_cats'] = X[cats].std(axis = 1)
    X['max_cats'] = np.amax(X[cats], axis = 1)
    
    X['sum_prods'] = X[prods].sum(axis = 1)
    X['mean_prods'] = X[prods].mean(axis = 1)
    X['std_prods'] = X[prods].std(axis = 1)
    X['max_prods'] = np.amax(X[prods], axis = 1)
    
    X['sum_change'] = X[change].sum(axis = 1)
    X['mean_change'] = X[change].mean(axis = 1)
    X['std_change'] = X[change].std(axis = 1)
    X['max_change'] = np.amax(X[change], axis = 1)
    
    return X


def prods_sums(X1):
    
    prods_names = []
    for i in change:
        j = i.replace('_ult1_change', '')
        prods_names.append(j)
    
    prods_groups = []
    for i, j in zip(prods, prods_names):
        prod = [x for x in prods if j in x]
        prods_groups.append(prod)

    X = X1.copy()
    sums = []
    for i, j in zip(prods_groups, prods_names):
        
        prod_sum = np.sum(X[i], axis = 1)
        X['{} sum'.format(j)] = prod_sum
        sums.append(prod_sum)
        
    """    
    sums = np.sum(sums, axis = 1).astype('int64')
    X_proportions = X.iloc[:, -24:].divide(sums, axis = 1)
    X = pd.concat([X, X_proportions], 1)
    """
    
    return X

In [ ]:
def age_clean(df):
    df.loc[df.age2 < 18,"age2"]  = df.loc[(df.age2 >= 18) & (df.age2 <= 30),"age2"].mean(skipna=True)
    df.loc[df.age2 > 100,"age2"] = df.loc[(df.age2 >= 30) & (df.age2 <= 100),"age2"].mean(skipna=True)
    df["age2"].fillna(df["age2"].mean(),inplace=True)
    df["age2"] = df["age2"].astype(int)
    
    return df

# Try to cut every 30 days and every 365

def cut_alta(df):
    alta_range = np.arange(0, np.max(df.fecha_alta), 30)
    df.fecha_alta = pd.cut(df.fecha_alta, alta_range, right=True)
    df.fecha_alta = pd.factorize(df.fecha_alta, sort = True)[0]
    return df

def cut_renta(df):
    renta_ranges = [0]+range(20000, 200001, 10000)
    renta_ranges += range(300000, 1000001, 100000)+[2000000, 100000000]
    df['renta'] = pd.cut(df.renta2, renta_ranges, right=True)
    df.renta = pd.factorize(df.renta, sort = True)[0]
    return df
# Try with sort and without

In [ ]:
X_tr2 = pd.concat([X_tr, add_tr, add_tr2], 1)
X_val2 = pd.concat([X_val, add_val, add_val2], 1)

X_tr2 = cut_renta(X_tr2)
X_val2 = cut_renta(X_val2)

X_tr2['renta2'] = np.log(X_tr2['renta2'])
X_val2['renta2'] = np.log(X_val2['renta2'])

X_tr2.drop(['indrel_1mes'], 1, inplace = True)
X_val2.drop(['indrel_1mes'], 1, inplace = True)

X_tr2 = age_clean(X_tr2)
X_val2 = age_clean(X_val2)

X_tr2 = cut_alta(X_tr2)
X_val2 = cut_alta(X_val2)


cats2 = [x for x in X_tr2.columns if 'ult1' not in x]
ohs = ['pais_residencia', 'segmento', 'canal_entrada', 'ind_empleado', 'nomprov']


X_tr2.shape

In [ ]:
X_test = pd.read_csv('Unused/test_full.csv')
test_prevs = pd.read_csv('test_previousproducts.csv').values

X_test.columns = clean + owned4 + owned3 + owned2 + owned1 + owned0

add_te = pd.read_csv('FULL_test_5lags.csv').loc[:, ['age', 'antiguedad', 'renta']]
add_te2 = pd.read_csv('FULL_test_5lags.csv').loc[:, ['fecha_alta']]

add_te.columns = ['age2', 'antiguedad2', 'renta2']


X_test.drop(orig_prods, 1, inplace = True)

X_test = pd.concat([X_test, add_te, add_te2], 1)
X_test = cut_renta(X_test)
X_test['renta2'] = np.log(X_test['renta2'])
X_test.drop(['indrel_1mes'], 1, inplace = True)
X_test = age_clean(X_test)
X_test = cut_alta(X_test)

X_test.shape

In [ ]:
from sklearn.preprocessing import PolynomialFeatures

cats2 = cats[:-2]

def prep_poly(X_tr, X_val, test = False, X_test = None):

    X_tr2 = add(X_tr)
    X_val2 = add(X_val)
    if test:
        X_test = add(X_test)

    poly = PolynomialFeatures(2)
    poly.fit(X_tr2[cats2])
    tr = pd.DataFrame(poly.transform(X_tr2[cats2]))
    val = pd.DataFrame(poly.transform(X_val2[cats2]))
    if test:
        te = pd.DataFrame(poly.transform(X_test[cats2]))
    
    X_tr2 = get_diffs(X_tr2)
    X_val2 = get_diffs(X_val2)
    print X_tr2.shape
    if test:
        X_test = get_diffs(X_test)

    X_tr3 = pd.concat([X_tr2, tr], 1)
    X_val3 = pd.concat([X_val2, val], 1)
    print X_tr3.shape
    if test:
        X_test = pd.concat([X_test, te], 1)
        
    if test:
        return X_tr3, X_val3, X_test
    
    return X_tr3, X_val3



#X_tr3, X_val3 = prep_poly(X_tr2, X_val2)

X_tr2, X_val2, X_test = prep_poly(X_tr2, X_val2, True, X_test)

print X_tr2.shape
print X_test.shape

In [ ]:
import itertools

comb4 = 'indext,indresi,ind_nuevo,segmento,indrel,ind_actividad_cliente,ind_empleado,' \
        'tiprel_1mes,antiguedad,fecha_alta,age,sexo,renta'.split(',')
    
comb1 = 'indext,indresi,ind_nuevo,segmento,indrel,ind_actividad_cliente,ind_empleado,' \
        'tiprel_1mes,antiguedad,fecha_alta'.split(',')

comb2 = 'segmento,age,ind_empleado,indfall,renta'.split(',')

comb3 = 'nomprov,canal_entrada,pais_residencia'.split(',')


def combs(X, one = False, ones_three = False, ones_four = False, two = False, two_three = False,
          three = False, three_three = False):
    
    X_tr = X.copy()
    
    if one:
        for comb in itertools.combinations(comb1, 2):
            feat = comb[0] + "_" + comb[1]
            X_tr[feat] = X_tr[comb[0]] + X_tr[comb[1]]
        print 'Comb1 interactions done.'
        
    if ones_three:
        for comb in itertools.combinations(comb1, 3):
            feat = comb[0] + "_" + comb[1] + "_" + comb[2]
            X_tr[feat] = X_tr[comb[0]] + X_tr[comb[1]] + X_tr[comb[2]]
        print '3-way Comb1 interactions done.'
    
    if ones_four:
    
        for comb in itertools.combinations(comb1, 4):
            feat = comb[0] + "_" + comb[1] + "_" + comb[2] + "_" + comb[3]
            X_tr[feat] = X_tr[comb[0]] + X_tr[comb[1]] + X_tr[comb[2]] + X_tr[comb[3]]
        print '4-way Comb1 interactions done.'
    
    if two:
        for comb in itertools.combinations(comb2, 2):
            feat = comb[0] + "_" + comb[1]
            X_tr[feat] = X_tr[comb[0]] + X_tr[comb[1]]
        print 'Comb2 interactions done.'
        
    if two_three:
        for comb in itertools.combinations(comb2, 3):
            feat = comb[0] + "_" + comb[1] + "_" + comb[2]
            X_tr[feat] = X_tr[comb[0]] + X_tr[comb[1]] + X_tr[comb[2]]
        print '3-way Comb2 interactions done.'
        
    if three:
        for comb in itertools.combinations(comb3, 2):
            feat = comb[0] + "_" + comb[1]
            X_tr[feat] = X_tr[comb[0]] + X_tr[comb[1]]
        print 'Comb3 interactions done.'
        
    if three_three:
        for comb in itertools.combinations(comb3, 3):
            feat = comb[0] + "_" + comb[1] + "_" + comb[2]
            X_tr[feat] = X_tr[comb[0]] + X_tr[comb[1]] + X_tr[comb[2]]
        print '3-way Comb3 interactions done.'
        
    print X_tr.shape
    return X_tr

X_tr2 = combs(X_tr2, True, True, False, True)
X_val2 = combs(X_val2, True, True, False, True)
X_test = combs(X_test, True, True, False, True)


X_tr2 = prods_sums(X_tr2)
X_val2 = prods_sums(X_val2)
X_test = prods_sums(X_test)

print X_tr2.shape
print X_val2.shape
print X_test.shape

In [ ]:
from sklearn.preprocessing import StandardScaler

def new(X_train, X_val, name, var1, var2):
    
    X_tr4 = X_train.copy()
    X_val4 = X_val.copy()
    
    X_tr4['{}'.format(name)] = (X_tr4.groupby(var1))[var2].transform('mean')
    X_val4['{}'.format(name)] = (X_val4.groupby(var1))[var2].transform('mean')
    
    return X_tr4, X_val4

def new_test(X_te, name, var1, var2):
    
    X_test = X_te.copy()
    X_test['{}'.format(name)] = (X_test.groupby(var1))[var2].transform('mean')
    
    return X_test


def add_cols(X_train, X_val, X_te = None):
    
    X_tr4 = X_train.copy()
    X_val4 = X_val.copy()
    if X_te is not None:
         X_test = X_te.copy()

    # by hand:
    X_tr4, X_val4 = new(X_tr4, X_val4, 'tiprel_by_actividad', 'ind_actividad_cliente', 'tiprel_1mes')
    X_tr4, X_val4 = new(X_tr4, X_val4, 'sum_change_by_renta', 'renta', 'sum_change')
    X_tr4, X_val4 = new(X_tr4, X_val4, 'sum_change_by_actividad', 'ind_actividad_cliente', 'sum_change')
    X_tr4, X_val4 = new(X_tr4, X_val4, 'sum_change_by_age', 'age', 'sum_change')
    X_tr4, X_val4 = new(X_tr4, X_val4, 'sum_change_by_sexo', 'sexo', 'sum_change')

    # from adder2:
    X_tr4, X_val4 = new(X_tr4, X_val4, 'sum_prods_groupingby_lags', lags2, 'sum_prods')
    X_tr4, X_val4 = new(X_tr4, X_val4, 'indext_groupingby_lags', lags2, 'indext')
    X_tr4, X_val4 = new(X_tr4, X_val4, 'ind_empleado_groupingby_lags', lags2 ,'ind_empleado')
    X_tr4, X_val4 = new(X_tr4, X_val4, 'nomprov_groupingby_lags', lags2, 'nomprov')


    # from adder3:
    X_tr4, X_val4 = new(X_tr4, X_val4, 'sum_prods_groupingby_change', change, 'sum_prods')
    X_tr4, X_val4 = new(X_tr4, X_val4, 'indext_groupingby_change', change, 'indext')
    
    #from adder_3way:
    X_tr4, X_val4 = new(X_tr4, X_val4, 'ind_actividad_cliente+segmento_groupingby_sum_change_mean2',
                        ['ind_actividad_cliente', 'segmento'], 'sum_change')
    X_tr4, X_val4 = new(X_tr4, X_val4, 'ind_actividad_cliente+segmento_groupingby_sum_prods_mean2',
                        ['ind_actividad_cliente', 'segmento'], 'sum_prods')
    X_tr4, X_val4 = new(X_tr4, X_val4, 'ind_actividad_cliente+tiprel_1mes_groupingby_fecha_alta_mean2',
                        ['ind_actividad_cliente', 'tiprel_1mes'], 'fecha_alta')
    
    
    
    if X_te is not None:
         # by hand:
        X_test = new_test(X_test, 'tiprel_by_actividad', 'ind_actividad_cliente', 'tiprel_1mes')
        X_test = new_test(X_test, 'sum_change_by_renta', 'renta', 'sum_change')
        X_test = new_test(X_test, 'sum_change_by_actividad', 'ind_actividad_cliente', 'sum_change')
        X_test = new_test(X_test, 'sum_change_by_age', 'age', 'sum_change')
        X_test = new_test(X_test, 'sum_change_by_sexo', 'sexo', 'sum_change')

        # from adder2:
        X_test = new_test(X_test, 'sum_prods_groupingby_lags', lags2, 'sum_prods')
        X_test = new_test(X_test, 'indext_groupingby_lags', lags2, 'indext')
        X_test = new_test(X_test, 'ind_empleado_groupingby_lags', lags2 ,'ind_empleado')
        X_test = new_test(X_test, 'nomprov_groupingby_lags', lags2, 'nomprov')


        # from adder3:
        X_test = new_test(X_test, 'sum_prods_groupingby_change', change, 'sum_prods')
        X_test = new_test(X_test, 'indext_groupingby_change', change, 'indext')
        
        #from adder_3way:
        X_test = new_test(X_test, 'ind_actividad_cliente+segmento_groupingby_sum_change_mean2',
                        ['ind_actividad_cliente', 'segmento'], 'sum_change')
        X_test = new_test(X_test, 'ind_actividad_cliente+segmento_groupingby_sum_prods_mean2',
                        ['ind_actividad_cliente', 'segmento'], 'sum_prods')
        X_test = new_test(X_test, 'ind_actividad_cliente+tiprel_1mes_groupingby_fecha_alta_mean2',
                        ['ind_actividad_cliente', 'tiprel_1mes'], 'fecha_alta')

    print X_tr4.shape
    print X_val4.shape
    
    if X_te is not None:
        print X_tr4.shape
        print X_val4.shape
        print X_test.shape
        
        return X_tr4, X_val4, X_test
    
    return X_tr4, X_val4


#X_tr4, X_val4 = add_cols(X_tr4, X_val4)

X_tr2, X_val2, X_test = add_cols(X_tr2, X_val2, X_test)

In [ ]:
from sklearn.preprocessing import StandardScaler, MinMaxScaler

ss = StandardScaler()
mm = MinMaxScaler()

prods_names = []
for i in change:
    j = i.replace('_ult1_change', '')
    prods_names.append(j)

prods_groups = []
for i, j in zip(prods, prods_names):
    prod = [x for x in prods if j in x]
    prods_groups.append(prod)
    

def new(X, name, var1, var2, func):
    
    X1 = X.copy()
    X1['{}'.format(name)] = (X1.groupby(var1))[var2].transform('{}'.format(func))
    
    return X1

def add_groups(X, var, func, concat = False, X2 = None):
    
    X1 = X.copy()

    X1 = new(X1, '{}_gr1'.format(var), prods_groups[0], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr2'.format(var), prods_groups[1], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr3'.format(var), prods_groups[2], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr4'.format(var), prods_groups[3], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr5'.format(var), prods_groups[4], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr6'.format(var), prods_groups[5], '{}'.format(var), '{}'.format(func))

    X1 = new(X1, '{}_gr7'.format(var), prods_groups[6], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr8'.format(var), prods_groups[7], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr9'.format(var), prods_groups[8], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr10'.format(var), prods_groups[9], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr11'.format(var), prods_groups[10], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr12'.format(var), prods_groups[11],  '{}'.format(var), '{}'.format(func))

    X1 = new(X1, '{}_gr13'.format(var), prods_groups[12], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr14'.format(var), prods_groups[13], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr15'.format(var), prods_groups[14], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr16'.format(var), prods_groups[15], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr17'.format(var), prods_groups[16], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr18'.format(var), prods_groups[17], '{}'.format(var), '{}'.format(func))

    X1 = new(X1, '{}_gr19'.format(var), prods_groups[18], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr20'.format(var), prods_groups[19], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr21'.format(var), prods_groups[20], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr22'.format(var), prods_groups[21], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr23'.format(var), prods_groups[22], '{}'.format(var), '{}'.format(func))
    X1 = new(X1, '{}_gr24'.format(var), prods_groups[23], '{}'.format(var), '{}'.format(func))
    
    cols = X1.iloc[:, -24:]
    if concat:
        X_f = pd.concat([X2, cols], 1)
        
        return X_f
    
    return cols


In [ ]:
X_tr3 = add_groups(X_tr2, 'sum_change', 'mean', True, X_tr2)
X_val3 = add_groups(X_val2, 'sum_change', 'mean', True, X_val2)

X_tr3 = add_groups(X_tr3, 'sum_prods', 'mean', True, X_tr3)
X_val3 = add_groups(X_val3, 'sum_prods', 'mean', True, X_val3)

print X_tr3.shape
print X_val3.shape


In [ ]:
import gc
del X_tr2, X_val2
gc.collect()

In [ ]:
X_test2 = add_groups(X_test, 'sum_change', 'mean', True, X_test)
X_test2 = add_groups(X_test2, 'sum_prods', 'mean', True, X_test2)
print X_test2.shape

del X_test
gc.collect()

In [ ]:
lgb_train = lgb.Dataset(X_tr3, y_tr)

# specify your configurations as a dict
params = {
    'task' : 'train',
    'boosting_type' : 'gbdt',
    'objective' : 'multiclass',
    'num_class': 24,
    'metric' : {'multi_logloss'},
    'num_leaves' : 181,
    'learning_rate' : 0.05,
    'feature_fraction' : 0.3762,
    'bagging_fraction': 0.9606,
    'min_data_in_leaf': 12,
    'bagging_freq': 100,
    'max_depth': 7,
    'silent': 1,
    'random_state': 2016,
    'verbose': 0,
}



print('Start training...')
gbm = lgb.train(params,
                lgb_train,
                num_boost_round = int(185/0.9),
                verbose_eval = 50)


#gbm.save_model('model.txt')

print('Start predicting...')
val_pred = gbm.predict(X_val3, num_iteration=gbm.best_iteration)
score = predict(val_pred, y_val_prevs, y_val_mapk, True)

In [ ]:
test_pred = gbm.predict(X_test2, num_iteration = gbm.best_iteration)
test_pred[:, 0] = 0
test_pred[:, 1] = 0

test_final = predict(test_pred, test_prevs)
test_sub = create_sub2(msg, test_final, 'LGBM_GroupedbyProdsGroupsFULL_21.11 MAP: {}'.format(score))

In [ ]:
from sklearn.model_selection import StratifiedKFold
import time

skf = StratifiedKFold(n_splits = 10, shuffle = True, random_state = 111)


def lgbm_skf(Xtr, Xval, predict_test = False):
    
    scores = []
    val_preds = []
    test_preds= []
    
    i = 1
    t = time.time()
    
    for train, test in skf.split(Xtr, y_tr):
        
        X_train, X_valid = Xtr.values[train], Xtr.values[test]
        y_train, y_valid = y_tr[train], y_tr[test]

        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

        # specify your configurations as a dict
        params = {
            'task' : 'train',
            'boosting_type' : 'gbdt',
            'objective' : 'multiclass',
            'num_class': 24,
            'metric' : {'multi_logloss'},
            'num_leaves' : 190,
            'learning_rate' : 0.05,
            'feature_fraction' : 0.4091,
            'bagging_fraction': 0.9922,
            'min_data_in_leaf': 14,
            'bagging_freq': 100,
            'max_depth': 7,
            'silent': 1,
            'random_state': 2016,
            'verbose': 0,
        }


        print 'Training on fold :', i
        gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round = 10000,
                        valid_sets = [lgb_train, lgb_eval], early_stopping_rounds = 15,
                        verbose_eval = False)

        val_pred = gbm.predict(Xval, num_iteration=gbm.best_iteration)
        val_pred[:, 0] = 0
        val_pred[:, 1] = 0
        
        if predict_test:
            test_pred = gbm.predict(X_test2, num_iteration = gbm.best_iteration)
            
        score = predict(val_pred, y_val_prevs, y_val_mapk, True)
        scores.append(score)
        
        val_preds.append(val_pred)
        if predict_test:
            test_preds.append(test_pred)
        i += 1
        
    val_preds = np.array(val_preds).mean(axis = 0)
    if predict_test:
        test_preds = np.array(test_preds).mean(axis = 0)
    
    final_score = predict(val_preds, y_val_prevs, y_val_mapk, True)
    
    print 'Mean score :', np.array(scores).mean()
    print 'Final MAP :', final_score
    print 'Time it took:', time.time() - t
    
    if predict_test:
        return val_preds, test_preds, final_score
    
    return val_preds

#val_preds = lgbm_skf(X_tr3, X_val3, False)

vals, tests, final_score = lgbm_skf(X_tr3, X_val3, True)

Final MAP : 0.859232472062
Mean score: 0.8555
Time it took: 471.421617985

In [ ]:
tests[:, 0] = 0
tests[:, 1] = 0

test_final = predict(tests, test_prevs)
test_sub = create_sub2(msg, test_final, 'LGBM_GroupedbyProdsGroups_21.11 MAP: {}'.format(final_score))

In [ ]:
def lgbm_skf_bo(bagging_fraction, feature_fraction, max_depth, min_data_in_leaf, num_leaves):
    
    scores = []
    val_preds = []
    test_preds= []
    
    i = 1
    
    for train, test in skf.split(X_tr2, y_tr):
        
        X_train, X_valid = X_tr2.values[train], X_tr2.values[test]
        y_train, y_valid = y_tr[train], y_tr[test]

        lgb_train = lgb.Dataset(X_train, y_train)
        lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)


        params = {
                'task' : 'train',
                'boosting_type' : 'gbdt',
                'objective' : 'multiclass',
                'num_class': 24,
                'metric' : {'multi_logloss'},
                'num_leaves' : num_leaves,
                'learning_rate' : 0.05,
                'feature_fraction' : feature_fraction,
                'bagging_fraction' :bagging_fraction,
                'min_data_in_leaf': min_data_in_leaf,
                'bagging_freq': 100,
                'max_depth': max_depth,
                'silent': 1,
                'random_state': 2016,
                'verbose': 0,
                }

        params['bagging_fraction'] = max(min(bagging_fraction, 1), 0)
        params['feature_fraction'] = max(min(feature_fraction, 1), 0)
        params['min_data_in_leaf'] = int(min_data_in_leaf)
        params['max_depth'] = int(max_depth)
        params['num_leaves'] = int(num_leaves)


        print 'Training on fold :', i
        gbm = lgb.train(params,
                        lgb_train,
                        num_boost_round = 10000,
                        valid_sets = [lgb_train, lgb_eval], early_stopping_rounds = 15,
                        verbose_eval = 100)

        val_pred = gbm.predict(X_val2, num_iteration=gbm.best_iteration)
        val_pred[:, 0] = 0
        val_pred[:, 1] = 0
        
            
        score = predict(val_pred, y_val_prevs, y_val_mapk, True)
        scores.append(score)
        
        val_preds.append(val_pred)
        i += 1
        
    val_preds = np.array(val_preds).mean(axis = 0)
    
    final_score = predict(val_preds, y_val_prevs, y_val_mapk, True)
    
    print 'Mean score :', np.array(scores).mean()
    print 'Final MAP :', final_score
    
    return final_score

Final MAP : 0.853861719695
   29 | 05m39s |    0.85386 |             0.9922 |             0.4091 |      7.4666 |            14.1727 |     189.6892 | 

In [ ]:
from bayes_opt import BayesianOptimization

lgbmBO2 = BayesianOptimization(lgbm_skf_bo, {
        'bagging_fraction': (0.9, 1),
        'feature_fraction': (0.35, 0.6),
        'max_depth': (5, 9),
        'min_data_in_leaf': (7, 25),
        'num_leaves': (170, 220),
    })

num_iter = 35
init_points = 20

lgbmBO2.maximize(init_points=init_points, n_iter=num_iter)

print('LGBM: %f' % lgbmBO2.res['max']['max_val'])

In [ ]:
lgb_train = lgb.Dataset(X_tr2, y_tr)

# specify your configurations as a dict
params = {
    'task' : 'train',
    'boosting_type' : 'gbdt',
    'objective' : 'multiclass',
    'num_class': 24,
    'metric' : {'multi_logloss'},
    'num_leaves' : 181,
    'learning_rate' : 0.05,
    'feature_fraction' : 0.3762,
    'bagging_fraction': 0.9606,
    'min_data_in_leaf': 12,
    'bagging_freq': 100,
    'max_depth': 7,
    'silent': 1,
    'random_state': 2016,
    'verbose': 0,
}



print('Start training...')
gbm = lgb.train(params,
                lgb_train,
                num_boost_round = int(185/0.9),
                verbose_eval = 50)


#gbm.save_model('model.txt')

print('Start predicting...')
val_pred = gbm.predict(X_val2, num_iteration=gbm.best_iteration)
score = predict(val_pred, y_val_prevs, y_val_mapk, True)

In [ ]:
test_preds = gbm.predict(X_test, num_iteration=gbm.best_iteration)
test_final = predict(test_preds, test_prevs)
test_sub = create_sub2(msg, test_final, 'LGBM_GroupedInteractions_20.11_FULL MAP: {}'.format(score))

In [ ]:
from dataset2 import Dataset
from average_precision import mapk
from genetic_search import genetic_search

def predict(preds, prevs, labels = None, validate = False):
    
    preds = preds
    filtered_rank = np.equal(prevs, 0) * preds
    predictions = np.argsort(filtered_rank, axis=1)
    predictions = predictions[:,::-1][:,0:7]
    
    if validate:
        score = mapk(labels, predictions)
        print "MAP:", score
        return score
    
    return predictions


dataset_root = '/home/w/DS_Projects/Kaggle/scripts/'
dataset = Dataset(dataset_root)

msg = {'train_month': [5],
       'eval_month': [16],
      'input_columns': [],
      'use_product': True,
      'use_change': True,
    }

def create_sub2(msg, preds, filename, verbose = True, save = True):
    
    test_month = 17
    msg['month'] = test_month
    predictions = preds
    #Create the submission text
    if verbose: print 'Creating text...'
    text='ncodpers,added_products\n'
    for i, ncodpers in enumerate(dataset.eval_current[dataset.eval_current.fecha_dato == test_month].ncodpers):
        text += '%i,' % ncodpers
        for j in predictions[i]:
            text += '%s ' % dataset.product_columns[j]
        text += '\n'
    #Write to file
    if verbose: print 'Writing to file...'
    if save:
        with gzip.open(dataset_root + 'submissions/%s.csv.gz' % filename, 'w') as f:
            f.write(text)
    
    return predictions

dictionary_types = {
                            "sexo":'category',
                            "ult_fec_cli_1t":str,
                            "indresi":'category',
                            "indext":'category',
                            "indrel":'category',
                            "indfall":'category',
                            "nomprov":'category',
                            "segmento":'category',
                            "ind_empleado":'category',
                            "pais_residencia":'category',
                            "indrel":'category',
                            "antiguedad":np.int16,
                            "ind_nuevo":'category',
                            'indrel_1mes':'category',
                            'tiprel_1mes':'category',
                            'canal_entrada':'category',
                            "age":np.int8,
                            "ind_actividad_cliente":'category',
                            "ind_ahor_fin_ult1":np.int8,
                            "ind_aval_fin_ult1":np.int8,
                            "ind_cco_fin_ult1":np.int8,
                            "ind_cder_fin_ult1":np.int8,
                            "ind_cno_fin_ult1":np.int8,
                            "ind_ctju_fin_ult1":np.int8,
                            "ind_ctma_fin_ult1":np.int8,
                            "ind_ctop_fin_ult1":np.int8,
                            "ind_ctpp_fin_ult1":np.int8,
                            "ind_deco_fin_ult1":np.int8,
                            "ind_deme_fin_ult1":np.int8,
                            "ind_dela_fin_ult1":np.int8,
                            "ind_ecue_fin_ult1":np.int8,
                            "ind_fond_fin_ult1":np.int8,
                            "ind_hip_fin_ult1":np.int8,
                            "ind_plan_fin_ult1":np.int8,
                            "ind_pres_fin_ult1":np.int8,
                            "ind_reca_fin_ult1":np.int8,
                            "ind_tjcr_fin_ult1":np.int8,
                            "ind_valo_fin_ult1":np.int8,
                            "ind_viv_fin_ult1":np.int8,
                            "ind_nomina_ult1":np.int8,
                            "ind_nom_pens_ult1":np.int8,
                            "ind_recibo_ult1":np.int8,

                            "ind_ahor_fin_ult1_change":'category',
                            "ind_aval_fin_ult1_change":'category',
                            "ind_cco_fin_ult1_change":'category',
                            "ind_cder_fin_ult1_change":'category',
                            "ind_cno_fin_ult1_change":'category',
                            "ind_ctju_fin_ult1_change":'category',
                            "ind_ctma_fin_ult1_change":'category',
                            "ind_ctop_fin_ult1_change":'category',
                            "ind_ctpp_fin_ult1_change":'category',
                            "ind_deco_fin_ult1_change":'category',
                            "ind_deme_fin_ult1_change":'category',
                            "ind_dela_fin_ult1_change":'category',
                            "ind_ecue_fin_ult1_change":'category',
                            "ind_fond_fin_ult1_change":'category',
                            "ind_hip_fin_ult1_change":'category',
                            "ind_plan_fin_ult1_change":'category',
                            "ind_pres_fin_ult1_change":'category',
                            "ind_reca_fin_ult1_change":'category',
                            "ind_tjcr_fin_ult1_change":'category',
                            "ind_valo_fin_ult1_change":'category',
                            "ind_viv_fin_ult1_change":'category',
                            "ind_nomina_ult1_change":'category',
                            "ind_nom_pens_ult1_change":'category',
                            "ind_recibo_ult1_change":'category',
                            'product_buy':np.int8, }

cols = []

for key, val in dictionary_types.iteritems():
    if val == 'category':
        cols.append(key)
        
cats = [x for x in cols if 'change' not in x]
changes = [x for x in cols if 'change' in x]
cats.append('antiguedad')
cats.append('age')

owned = []

for key, val in dictionary_types.iteritems():
    if val == np.int8:
        owned.append(key)
        
owned.remove('age')
owned.remove('product_buy')

full = cats + owned + changes

owned4 = []
owned3 = []
owned2 = []
owned1 = []
owned0 = []

for i in owned:
    j = i + "_4"
    owned4.append(j)

for i in owned:
    j = i + "_3"
    owned3.append(j)

for i in owned:
    j = i + "_2"
    owned2.append(j)

for i in owned:
    j = i + "_1"
    owned1.append(j)

for i in owned:
    j = i + "_0"
    owned0.append(j)

    
clean = full[0:16] + dataset.product_columns + dataset.change_columns